# 1. Data acquisition

donlowad datasets from their sources

get all datasets into xyz format (lon, lat, observabble)

Rename columns into lon lat observable

make sure that longitude is in -180 - 180 format

remove missing latitudes and longitues

save xyz files into xyz folder

# Import necessary libraries 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from agrid.grid import Grid
from pathlib import Path
import os, sys, pickle

from pathlib import Path

import pandas as pd

import re
import copy

import numpy as np
from numpy import asarray
from numpy import save
from numpy import load

from scipy import stats, interpolate, spatial, io
from scipy.ndimage import gaussian_filter, median_filter
import xarray as xr
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.collections import LineCollection
import matplotlib.colors as mcolors
import matplotlib as mpl
from matplotlib.patches import Arc 
import cartopy.crs as ccrs

import pyproj as proj
import rasterio


from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error

import seaborn as sns

from joblib import Parallel, delayed
import multiprocessing
import numba as nb
from numba import jit

import warnings
warnings.filterwarnings('ignore')

In [3]:
# constants
#parent directory

dir_p = Path().resolve() 

#constants
km = 1000
milli = 0.001
micro = 0.000001

# fig size for presentation
fig_pres_small = (4,3)

fig_pres_small_cbar = (4,2)
#aspect ratio
fig_pres_large = (16,9)



In [4]:

# We can exclude Arctic ocean and Antarctica, as there are no HF measurements to use
world_lon_min, world_lon_max, world_lat_min, world_lat_max  = -180, 180, -60, 80

# map extents of Africa and Australia
afr_lon_min, afr_lon_max, afr_lat_min, afr_lat_max =  -20, 52, -37 , 38  



# create grid for each region
# crs Coordinate reference system

#EPSG is projection
# 0.2 degrees equal roughly 20 km

World = Grid(res=[0.2, 0.2], up=world_lat_max, down=world_lat_min)

# africa grid

Africa =    Grid(res=[0.2, 0.2],  left = afr_lon_min, right= afr_lon_max, up=afr_lat_max , down=afr_lat_min)

# africa grid low resolution 50 x 50 km

Africa_50 =    Grid(res=[0.5, 0.5],  left = afr_lon_min, right= afr_lon_max, up=afr_lat_max , down=afr_lat_min)


#dictionary of all grids

grids = {}

grids['Afr'] = Africa
grids['Afr_50'] = Africa_50
grids['World'] = World


create dictionaries to ease looping

In [5]:
# to ease looping with dictionaries

regions_a_a5 = [ 'Afr','Afr_50' ]

regions_w_a = [ 'World' ,'Afr',]

regions_w_a= [ 'World' ,'Afr']

regions_a_a5 = [ 'Afr', 'Afr_50']

regions_w_a_a5 = [ 'World' ,'Afr', 'Afr_50']



regions_Total = ['World' ,'Afr', 'Afr_50',]



# raster exenets to adjust map
raster_extent_Afr = [grids['Afr'].extent[0], grids['Afr'].extent[1], grids['Afr'].extent[3], grids['Afr'].extent[2]]
raster_extent_Afr_50 = [grids['Afr_50'].extent[0], grids['Afr_50'].extent[1], grids['Afr_50'].extent[3], grids['Afr_50'].extent[2]]
raster_extent_World = [grids['World'].extent[0], grids['World'].extent[1], grids['World'].extent[3], grids['World'].extent[2]]

# to correct plot maps
raster_extents = {}

raster_extents['Afr'] = raster_extent_Afr
raster_extents['Afr_50'] = raster_extent_Afr_50
raster_extents['World'] = raster_extent_World


# list of latitudes and longitudes
lon_dict = {}
lat_dict = {}

lon_dict['Afr'] = [afr_lon_min, afr_lon_max]
lon_dict['Afr_50'] = [afr_lon_min, afr_lon_max]
lon_dict['World'] = [world_lon_min, world_lon_max]

lat_dict['Afr'] = [afr_lat_min, afr_lat_max]
lat_dict['Afr_50'] = [afr_lat_min, afr_lat_max]
lat_dict['World'] = [world_lat_min, world_lat_max]



import datasets and transform them

In [6]:
# sv and pv calcualtion for africa only once


seismic_w_sv_i_f = dir_p / 'data'/'dataset'/'Reference'/'d_Shear_Wave_Speed'/'SL2013sv_dVs_25km-0.5d_50+.xyz'
seismic_rel_w_sv_xyz_o_f = dir_p / 'data'/'dataset'/'XYZ'/'d_Shear_Wave_Speed'/'SV_W_150.xyz'

seismic_sv_i_f = dir_p / 'data'/'dataset'/'Reference'/'d_Shear_Wave_Speed'/'AF2019_vsvp.mod'
seismic_rel_a_sv_xyz_o_f  = dir_p / 'data'/'dataset'/'XYZ'/'d_Shear_Wave_Speed'/'SV_A_150.xyz'
seismic_a_sv_xyz_o_f  = dir_p / 'data'/'dataset'/'XYZ'/'d_Shear_Wave_Speed'/'SV_A_150_speed.xyz'


seismic_w_pv_i_f = dir_p / 'data'/'dataset'/'Reference'/'e_Pressure_Wave_Speed'/'DETOX-P1.txt'
seismic_rel_w_pv_xyz_o_f = dir_p / 'data'/'dataset'/'XYZ'/'e_Pressure_Wave_Speed'/'PV_W_150.xyz'


seismic_a_pv_i_f = dir_p / 'data'/'dataset'/'Reference'/'e_Pressure_Wave_Speed'/'AFRP20_RF_CR1_MOD.txt'
seismic_rel_a_pv_xyz_o_f = dir_p / 'data'/'dataset'/'XYZ'/'e_Pressure_Wave_Speed'/'PV_A_150.xyz'
seismic_a_pv_xyz_o_f = dir_p / 'data'/'dataset'/'XYZ'/'e_Pressure_Wave_Speed'/'PV_A_150_speed.xyz'

# Read geo data

#is data -180 to 180

# SV input world



init_df = pd.read_csv(seismic_w_sv_i_f,header=None, skiprows=1, skip_blank_lines=True, 
                      na_filter=True, delim_whitespace=True)

print(f'before z: {init_df[5].max()}')
print(f'before lon : {init_df[1].min()}')
init_df[5] = init_df[5].multiply(0.01)
print(f'after z: {init_df[5].max()}')
print(f'after lon : {init_df[1].min()}')
print('#####')

seismic_w_sv_df_150_df = init_df[init_df[0]==150][[1,2,5]]
#seismic_w_sv_df = copy.deepcopy(seismic_w_sv_df_150_df.iloc[:,[1,2,5]]


seismic_w_sv_df_150_df.columns = ['lon', 'lat', 'dVs(%)']

    
seismic_w_sv_df_150_df['dVs(%)'] = round(seismic_w_sv_df_150_df['dVs(%)'],4)

seismic_w_sv_df_150_df.replace(' ', np.nan, inplace=True)


seismic_w_sv_df_150_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_w_sv_df_150_df.reset_index( inplace=True,drop=True)


seismic_w_sv_df_150_df.to_csv(seismic_rel_w_sv_xyz_o_f, header=True, 
                              index=False,  sep='\t')


#######
# SV africa


seismic_a_sv_df = pd.read_csv(seismic_sv_i_f   , skip_blank_lines=True, 
                      na_filter=True, delim_whitespace=True)


print(f'before Z : {seismic_a_sv_df.iloc[:, 2].max()}')
print(f'lon : {seismic_a_sv_df.iloc[:, 0].min()}')

seismic_a_sv_df[9] = seismic_a_sv_df.iloc[:,8] / seismic_a_sv_df.iloc[:,7]
print(f'after Z : {seismic_a_sv_df.iloc[:,9] .max()}')
print('#####')

seismic_a_sv_final_df = seismic_a_sv_df[seismic_a_sv_df.iloc[:,2]==150].iloc[:,[0,1 , 9]]


seismic_a_sv_final_df.columns = ['lon', 'lat', 'dVs(%)']


    
seismic_a_sv_final_df['dVs(%)'] = round(seismic_a_sv_final_df['dVs(%)'],4)    
   
seismic_a_sv_final_df.replace(' ', np.nan, inplace=True)
seismic_a_sv_final_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_a_sv_final_df.reset_index(inplace=True,drop=True)


seismic_a_sv_final_df.to_csv(seismic_rel_a_sv_xyz_o_f, index=False, header=True, sep='\t')

#######
# SV africa speed


seismic_a_sv_df = pd.read_csv(seismic_sv_i_f   , skip_blank_lines=True, 
                      na_filter=True, delim_whitespace=True)


print(f'before Z : {seismic_a_sv_df.iloc[:, 2].max()}')
print(f'lon : {seismic_a_sv_df.iloc[:, 0].min()}')


seismic_a_sv_final_df = seismic_a_sv_df[seismic_a_sv_df.iloc[:,2]==150].iloc[:,[0,1 , 6]]


seismic_a_sv_final_df.columns = ['lon', 'lat', 'Vs(Km/s)']


    
seismic_a_sv_final_df['Vs(Km/s)'] = round(seismic_a_sv_final_df['Vs(Km/s)'],4)/1000   


   
seismic_a_sv_final_df.replace(' ', np.nan, inplace=True)
seismic_a_sv_final_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_a_sv_final_df.reset_index(inplace=True,drop=True)


seismic_a_sv_final_df.to_csv(seismic_a_sv_xyz_o_f, index=False, header=True, sep='\t')



#######
#PV world

seismic_w_pv_df = pd.read_csv(seismic_w_pv_i_f, skiprows=3,
                              header=None, engine='python', skip_blank_lines=True,   na_filter=True, delim_whitespace=True)



print(f'before Z : {seismic_w_pv_df[2] .max()}')
print(f'lon : {seismic_w_pv_df.iloc[:,0].min()}')
seismic_w_pv_df[2] = seismic_w_pv_df.iloc[:,2].multiply(0.01)
print(f'after Z : {seismic_w_pv_df.iloc[:,2] .max()}')
print(f'after lon : { seismic_w_pv_df.iloc[:,0].min()}')
print('#####')

seismic_w_pv_df.replace(' ', np.nan, inplace=True)

seismic_w_pv_df.columns = ['lon', 'lat', 'dVp(%)']


if seismic_w_pv_df.iloc[:,0].min() >= 0 :
    print(f'true {seismic_w_pv_df.iloc[:,0].min()}')
    
    mask = seismic_w_pv_df.iloc[:,0]> 180
    seismic_w_pv_df.loc[mask,'lon'] -= 360
    
seismic_w_pv_df['dVp(%)'] = round(seismic_w_pv_df['dVp(%)'],4)


seismic_w_pv_df_final = seismic_w_pv_df[['lon', 'lat', 'dVp(%)']]
seismic_w_pv_df_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_w_pv_df_final.reset_index( inplace=True,drop=True)

seismic_w_pv_df_final.to_csv(seismic_rel_w_pv_xyz_o_f, header=True, index=False, sep='\t')



#####

#PV africa

seismic_a_pv_df = pd.read_csv(seismic_a_pv_i_f, header=None ,sep='\s+',  
                               engine='python', skip_blank_lines=True,   na_filter=True)

seismic_a_pv_150_df = seismic_a_pv_df[seismic_a_pv_df.iloc[:,0]==150][[2,1,3]]
#seismic_a_pv_150_copy_df = copy.deepcopy(seismic_a_pv_150_df.iloc[:, [2,1,3]])

print(f'before Z : {seismic_a_pv_150_df[2] .max()}')
print(f'lon : {seismic_a_pv_150_df[1] .min()}')
seismic_a_pv_150_df[3] = seismic_a_pv_150_df[3].multiply(0.01)
print(f'after Z : {seismic_a_pv_150_df[2] .max()}')
print('#####')

seismic_a_pv_150_df.columns =  ['lon', 'lat', 'dVp(%)']

    
seismic_a_pv_150_df['dVp(%)'] = round(seismic_a_pv_150_df['dVp(%)'],4)



seismic_a_pv_150_df.replace(' ', np.nan, inplace=True)
seismic_a_pv_150_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_a_pv_150_df.reset_index( inplace=True,drop=True)

seismic_a_pv_150_df.to_csv(seismic_rel_a_pv_xyz_o_f, header=True, index=False, sep='\t')


########


#PV africa spped

seismic_a_pv_df = pd.read_csv(seismic_a_pv_i_f, header=None ,sep='\s+',  
                               engine='python', skip_blank_lines=True,   na_filter=True)

seismic_a_pv_150_df = seismic_a_pv_df[seismic_a_pv_df.iloc[:,0]==150][[2,1,4]]
#seismic_a_pv_150_copy_df = copy.deepcopy(seismic_a_pv_150_df.iloc[:, [2,1,3]])


seismic_a_pv_150_df.columns =  ['lon', 'lat', 'Vp(Km/s)']

    
seismic_a_pv_150_df['Vp(Km/s)'] = round(seismic_a_pv_150_df['Vp(Km/s)'],4)



seismic_a_pv_150_df.replace(' ', np.nan, inplace=True)

seismic_a_pv_150_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
seismic_a_pv_150_df.reset_index( inplace=True,drop=True)

seismic_a_pv_150_df.to_csv(seismic_a_pv_xyz_o_f, header=True, index=False, sep='\t')


print('terminated')

before z: 10.7214
before lon : -180.0
after z: 0.10721399999999999
after lon : -180.0
#####
before Z : 660.0
lon : -25.0
after Z : 0.10001935359009095
#####
before Z : 660.0
lon : -25.0
before Z : 2.48
lon : 0.0
after Z : 0.0248
after lon : 0.0
#####
true 0.0
before Z : 64.0
lon : -44.0
after Z : 64.0
#####
terminated


In [7]:

CTD_Gard_Hasterok_f = dir_p / 'data'/'dataset'/'Reference'/'f_CTD'/'Gard&Hasterok2021.txt'
CTD_Gard_Hasterok_xyz_o = dir_p / 'data'/'dataset'/'XYZ'/'f_CTD'/'CTD.xyz'


REG_f = dir_p / 'data'/'dataset'/'Reference'/'o_Tectonic_Regionalization_Class'/'SL2013sv_Cluster_2d'
REG_xyz_o = dir_p / 'data'/'dataset'/'XYZ'/'o_Tectonic_Regionalization_Class'/'REG.xyz'



CTD_gh_df = pd.read_csv(CTD_Gard_Hasterok_f ,header=None,   delim_whitespace=True)
print(f'min z: {CTD_gh_df.iloc[:,2].min()}')
print(f'min lon : {CTD_gh_df.iloc[:,0].min()}')
print(f'max z: {CTD_gh_df.iloc[:,2].max()}')
print(f'max lon : {CTD_gh_df.iloc[:,0].max()}')
CTD_gh_df.columns =  ['lon', 'lat', 'ctd']


CTD_gh_df['ctd'] = round(CTD_gh_df['ctd'],4)

CTD_gh_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
CTD_gh_df.reset_index( inplace=True,drop=True)

assert CTD_gh_df['lon'].max() <= 180, f" max lon {CTD_gh_df['lon'].max()}"
assert CTD_gh_df['lon'].min() >= -180, f" min lon {CTD_gh_df['lon'].min()}"
assert CTD_gh_df['lat'].max() <= 90, f" max lat {CTD_gh_df['lat'].max()}"
assert CTD_gh_df['lat'].min() >= -90, f" min lat {CTD_gh_df['lon'].min()}"

CTD_gh_df .to_csv(CTD_Gard_Hasterok_xyz_o , index=False, header=True, sep='\t')




reg_df = pd.read_csv(REG_f ,header=None,   delim_whitespace=True)
print(f'min z: {reg_df.iloc[:,2].min()}')
print(f'min lon : {reg_df.iloc[:,0].min()}')
print(f'max z: {reg_df.iloc[:,2].max()}')
print(f'max lon : {reg_df.iloc[:,0].max()}')
reg_df.columns =  ['lon', 'lat', 'classes']
  


reg_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
reg_df.reset_index( inplace=True,drop=True)

assert reg_df['lon'].max() <= 180, f" max lon {reg_df['lon'].max()}"
assert reg_df['lon'].min() >= -180, f" min lon {reg_df['lon'].min()}"
assert reg_df['lat'].max() <= 90, f" max lat {reg_df['lat'].max()}"
assert reg_df['lat'].min() >= -90, f" min lat {reg_df['lon'].min()}"

reg_df.to_csv(REG_xyz_o , index=False, header=True, sep='\t')

print('terminated')


min z: -7.4117378
min lon : -179.5
max z: 160.59855
max lon : 179.5
min z: 1
min lon : -180.0
max z: 6
max lon : 180.0
terminated


In [8]:

MOHO_new_global_f = dir_p / 'data'/'dataset'/'Reference'/'a_Moho_Depth'/'New_Global_Moho.DAT'
MOHO_new_global_xyz_o = dir_p / 'data'/'dataset'/'XYZ'/'a_Moho_Depth'/'MOHO.xyz'



######

moho_ng_df = pd.read_csv(MOHO_new_global_f ,header=None, skiprows=1, engine='python', 
                         encoding="ISO-8859-1", delim_whitespace=True)

print(f'min z: {moho_ng_df .iloc[:,2].min()}')
print(f'min lon : {moho_ng_df .iloc[:,0].min()}')
print(f'max z: {moho_ng_df .iloc[:,2].max()}')
print(f'max lon : {moho_ng_df .iloc[:,0].max()}')
moho_ng_df.columns =  ['lon', 'lat', 'Moho_depth']

    
if moho_ng_df.iloc[:,0].min() >= 0 :
    print(f'true {moho_ng_df.iloc[:,0].min()}')
    
    mask = moho_ng_df.iloc[:,0]> 180
    moho_ng_df.loc[mask,'lon'] -= 360
    
print(f'after lon min {moho_ng_df["lon"].min()}')

moho_ng_df['Moho_depth'] = round(moho_ng_df['Moho_depth'],4)


moho_ng_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
moho_ng_df.reset_index( inplace=True,drop=True)


assert moho_ng_df['lon'].max() <= 180, f" max lon {moho_ng_df['lon'].max()}"
assert moho_ng_df['lon'].min() >= -180, f" min lon {moho_ng_df['lon'].min()}"
assert moho_ng_df['lat'].max() <= 90, f" max lat {moho_ng_df['lat'].max()}"
assert moho_ng_df['lat'].min() >= -90, f" min lat {moho_ng_df['lon'].min()}"
moho_ng_df.to_csv(MOHO_new_global_xyz_o , index=False, header=True, sep='\t')



print('terminated')

min z: 6.9941
min lon : 0.0
max z: 70.9321
max lon : 359.0
true 0.0
after lon min -179.0
terminated


In [9]:
curveture_f = dir_p / 'data' /'dataset'/ 'Reference'/'n_Shape_Index_Curvature' / 'GOCE_Curvature_Topo_Iso_Corr.txt'
curveture_xyz_o = dir_p / 'data' /'dataset'/ 'XYZ'/'n_Shape_Index_Curvature' / 'SI.xyz'



si_df = pd.read_csv(curveture_f ,header=None,  engine='python',skiprows=1,  encoding="ISO-8859-1", delim_whitespace=True)
print(f'min z: {si_df .iloc[:,2].min()}')
print(f'min lon : {si_df .iloc[:,0].min()}')
print(f'max lon : {si_df .iloc[:,0].max()}')
print(f'max z : {si_df .iloc[:,2].max()}')
SI_final = si_df[[0,1,2]]




SI_final.columns =  ['lon', 'lat', 'SI']
if SI_final['lon'].min() >= 0 :
    SI_final['lon'] = SI_final['lon'] - 180

SI_final['SI'] = round(SI_final['SI'],4)

mask_gt = SI_final['lon'] > 180
mask_lt = SI_final['lon'] < -180

mask_gt_d = si_df[0] > 180
mask_lt_d = si_df[0] < -180


print(len(SI_final))
print(f'Outlier max {mask_gt.value_counts()}')
print(f'outlier min {mask_lt.value_counts()}')
#print(f'max outlier vlaue {SI_final.loc[SI_final[mask_gt].index,:]}')
#print(f'min outlier value {SI_final.loc[SI_final[mask_lt].index,:]}')
print(f'max outlier vlaue {si_df.loc[si_df[mask_gt_d].index,:]}')
print(f'min outlier value {si_df.loc[si_df[mask_lt_d].index,:]}')

SI_final.drop( SI_final[mask_gt].index , inplace=True)
SI_final.drop( SI_final[mask_lt].index , inplace=True)

assert SI_final['lon'].max() <= 180, f" max lon {SI_final['lon'].max()}"
assert SI_final['lon'].min() >= -180, f" min lon {SI_final['lon'].min()}"
assert SI_final['lat'].max() <= 90, f" max lat {SI_final['lat'].max()}"
assert SI_final['lat'].min() >= -90, f" min lat {SI_final['lon'].min()}"

SI_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
SI_final.reset_index( inplace=True,drop=True)

SI_final.to_csv(curveture_xyz_o , index=False, header=True, sep='\t')


print('terminated')

min z: -0.99939
min lon : -180.0
max lon : 949.0
max z : 0.99932
1618202
Outlier max False    1618201
True           1
Name: lon, dtype: int64
outlier min False    1618202
Name: lon, dtype: int64
max outlier vlaue              0        1         2         3         4      5         6
1305745  949.0  0.00814  0.007882 -0.000396  0.012604  0.013  0.006091
min outlier value Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6]
Index: []
terminated


In [10]:

LithoRef_f  = dir_p / 'data' /'dataset'/ 'Reference'/'b_LAB' / 'LithoRef18.xyz'

rho_c_xyz_o =  dir_p / 'data' /'dataset'/ 'XYZ'/'j_Crustal_Average_Density'/'RHO_C.xyz'
rho_l_xyz_o =  dir_p / 'data' /'dataset'/ 'XYZ'/'i_Lithosphere_Average_Density'/'RHO_L.xyz'

LAB_NG_f =  dir_p / 'data' /'dataset'/ 'Reference'/'b_LAB'/'New_Global_LAB.DAT'
LAB_NG_xyz_o =  dir_p / 'data' /'dataset'/ 'XYZ'/'b_LAB'/'LAB.xyz'




lith_df = pd.read_csv(LithoRef_f ,header=None,  engine='python',skiprows=9,  encoding="ISO-8859-1", delim_whitespace=True)
print(f'min z: {lith_df .iloc[:,2].min()}')
print(f'min lon : {lith_df .iloc[:,0].min()}')
print(f'max z: {lith_df.iloc[:,2].max()}')
print(f'max lon : {lith_df .iloc[:,0].max()}')

lab_final = lith_df.iloc[:,[0,1,4]]
lab_final.columns = ['lon', 'lat', 'LAB_depth']



lab_final['LAB_depth'] = round(lab_final['LAB_depth'],4)

lab_final['LAB_depth'] = lab_final['LAB_depth']/(-km)
lab_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
lab_final.reset_index( inplace=True,drop=True)

rho_c_final = lith_df.iloc[:,[0,1,5]]
rho_c_final.columns = ['lon', 'lat', 'RHO_C']



rho_c_final['RHO_C'] = round(rho_c_final['RHO_C'],4)

rho_c_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
rho_c_final.reset_index( inplace=True,drop=True)

rho_l_final = lith_df.iloc[:,[0,1,6]]
rho_l_final.columns = ['lon', 'lat', 'RHO_L']


rho_l_final['RHO_L'] = round(rho_l_final['RHO_L'],4)

rho_l_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
rho_l_final.reset_index( inplace=True,drop=True)

assert lab_final['lon'].max() <= 180, f" max lon {lab_final['lon'].max()}"
assert lab_final['lon'].min() >= -180, f" min lon {lab_final['lon'].min()}"
assert lab_final['lat'].max() <= 90, f" max lat {lab_final['lat'].max()}"
assert lab_final['lat'].min() >= -90, f" min lat {lab_final['lon'].min()}"

assert rho_c_final['lon'].max() <= 180, f" max lon {rho_c_final['lon'].max()}"
assert rho_c_final['lon'].min() >= -180, f" min lon {rho_c_final['lon'].min()}"
assert rho_c_final['lat'].max() <= 90, f" max lat {rho_c_final['lat'].max()}"
assert rho_c_final['lat'].min() >= -90, f" min lat {rho_c_final['lon'].min()}"

assert rho_l_final['lon'].max() <= 180, f" max lon {rho_l_final['lon'].max()}"
assert rho_l_final['lon'].min() >= -180, f" min lon {rho_l_final['lon'].min()}"
assert rho_l_final['lat'].max() <= 90, f" max lat {rho_l_final['lat'].max()}"
assert rho_l_final['lat'].min() >= -90, f" min lat {rho_l_final['lon'].min()}"


rho_c_final.to_csv(rho_c_xyz_o , index=False, header=True, sep='\t')
rho_l_final.to_csv(rho_l_xyz_o , index=False, header=True, sep='\t')

LAB_NG_df = pd.read_csv(LAB_NG_f ,header=None,  engine='python',skiprows=1,  encoding="ISO-8859-1", delim_whitespace=True)
print(f'min z: {LAB_NG_df .iloc[:,2].min()}')
print(f'min lon : {LAB_NG_df .iloc[:,0].min()}')
print(f'len: {len(LAB_NG_df)}')
print(f'max z: {LAB_NG_df .iloc[:,2].max()}')
print(f'max lon : {LAB_NG_df .iloc[:,0].max()}')
print(f'len: {len(LAB_NG_df)}')
LAB_NG_df.columns = ['lon', 'lat', 'LAB_depth']

if LAB_NG_df.iloc[:,0].min() >= 0 :
    print(f'true {LAB_NG_df.iloc[:,0].min()}')
    
    mask = LAB_NG_df.iloc[:,0]> 180
    LAB_NG_df.loc[mask,'lon'] -= 360
    
    
print(f'after {LAB_NG_df["lon"].min()}')

LAB_NG_df['LAB_depth'] = round(LAB_NG_df['LAB_depth'],4)

LAB_NG_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
LAB_NG_df.reset_index( inplace=True,drop=True)

assert LAB_NG_df['lon'].max() <= 180, f" max lon {LAB_NG_df['lon'].max()}"
assert LAB_NG_df['lon'].min() >= -180, f" min lon {LAB_NG_df['lon'].min()}"
assert LAB_NG_df['lat'].max() <= 90, f" max lat {LAB_NG_df['lat'].max()}"
assert LAB_NG_df['lat'].min() >= -90, f" min lat {LAB_NG_df['lon'].min()}"

LAB_NG_df.to_csv(LAB_NG_xyz_o , index=False, header=True, sep='\t')

print('terminated')

min z: -6540.0
min lon : -179.0
max z: 5320.0
max lon : 179.0
min z: 45.0
min lon : 0.0
len: 12232
max z: 300.0
max lon : 359.0
len: 12232
true 0.0
after -179.0
terminated


In [11]:
geoid_f = dir_p / 'data' /'dataset'/ 'Reference'/'l_geoid_height' / 'EIGEN-6C4_geoid.gdf'
geoid_xyz_o = dir_p / 'data' /'dataset'/ 'XYZ'/'l_geoid_height' / 'EIGEN-6C4_geoid.xyz'

FA_f = dir_p / 'data' /'dataset'/ 'Reference'/'k_Free_Air' / 'EIGEN-6C4_gravity_anomaly_cl.gdf'
FA_xyz_o = dir_p / 'data' /'dataset'/ 'XYZ'/'k_Free_Air' / 'EIGEN-6C4_gravity_anomaly_cl.xyz'

bg_f = dir_p / 'data' /'dataset'/ 'Reference'/'m_Bouguer' / 'EIGEN-6C4_gravity_anomaly_bg.gdf'
bg_xyz_o = dir_p / 'data' /'dataset'/ 'XYZ'/'m_Bouguer' / 'EIGEN-6C4_gravity_anomaly_bg.xyz'



geoid_df = pd.read_csv(geoid_f ,header=None,  engine='python',skiprows=36,  encoding="ISO-8859-1", delim_whitespace=True)

print(f'min z: {geoid_df.iloc[:,2].min()}')
print(f'min lon : {geoid_df.iloc[:,0].min()}')
print(f'max z: {geoid_df.iloc[:,2].max()}')
print(f'max lon : {geoid_df.iloc[:,0].max()}')

geoid_df.columns = ['lon', 'lat', 'mgal']


geoid_df = geoid_df.round(4)


geoid_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
geoid_df.reset_index( inplace=True,drop=True)

assert geoid_df['lon'].max() <= 180, f" max lon {geoid_df['lon'].max()}"
assert geoid_df['lon'].min() >= -180, f" min lon {geoid_df['lon'].min()}"
assert geoid_df['lat'].max() <= 90, f" max lat {geoid_df['lat'].max()}"
assert geoid_df['lat'].min() >= -90, f" min lat {geoid_df['lon'].min()}"

geoid_df.to_csv(geoid_xyz_o , index=False, header=True, sep='\t')


#####
FA_df = pd.read_csv(FA_f ,header=None,  engine='python',skiprows=36,  encoding="ISO-8859-1", delim_whitespace=True)


print(f'min z: {FA_df .iloc[:,2].min()}')
print(f'min lon : {FA_df .iloc[:,0].min()}')
print(f'max z: {FA_df .iloc[:,2].max()}')
print(f'max lon : {FA_df .iloc[:,0].max()}')

FA_df.columns = ['lon', 'lat', 'mgal']

FA_df = FA_df.round(4)


FA_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
FA_df.reset_index( inplace=True,drop=True)

assert FA_df['lon'].max() <= 180, f" max lon {FA_df['lon'].max()}"
assert FA_df['lon'].min() >= -180, f" min lon {FA_df['lon'].min()}"
assert FA_df['lat'].max() <= 90, f" max lat {FA_df['lat'].max()}"
assert FA_df['lat'].min() >= -90, f" min lat {FA_df['lon'].min()}"

FA_df.to_csv(FA_xyz_o , index=False, header=True, sep='\t')

#####


bg_df = pd.read_csv(bg_f ,header=None,  engine='python',skiprows=37,  encoding="ISO-8859-1", delim_whitespace=True)


print(f'min z: {bg_df.iloc[:,2].min()}')
print(f'min lon : {bg_df.iloc[:,0].min()}')
print(f'max z: {bg_df.iloc[:,2].min()}')
print(f'max lon : {bg_df.iloc[:,0].max()}')

bg_df.columns = ['lon', 'lat', 'mgal']


bg_df = bg_df.round(4)


bg_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
bg_df.reset_index( inplace=True,drop=True)

assert bg_df['lon'].max() <= 180, f" max lon {bg_df['lon'].max()}"
assert bg_df['lon'].min() >= -180, f" min lon {bg_df['lon'].min()}"
assert bg_df['lat'].max() <= 90, f" max lat {bg_df['lat'].max()}"
assert bg_df['lat'].min() >= -90, f" min lat {bg_df['lon'].min()}"

bg_df.to_csv(bg_xyz_o , index=False, header=True, sep='\t')

print('terminated')

min z: -106.184427114513
min lon : -180.0
max z: 84.917290827278
max lon : 180.0
min z: -301.300980351904
min lon : -180.0
max z: 522.25583386214
max lon : 180.0
min z: -631.218235051554
min lon : -180.0
max z: -631.218235051554
max lon : 180.0
terminated


In [12]:
REG_f = dir_p / 'data'/'dataset'/'Reference'/'o_Tectonic_Regionalization_Class'/'SL2013sv_Cluster_2d'
REG_o = dir_p / 'data'/'dataset'/'Reference'/'o_Tectonic_Regionalization_Class'/'REG.xyz'
REG_xyz_o = dir_p / 'data'/'dataset'/'XYZ'/'o_Tectonic_Regionalization_Class'/'REG.xyz'




reg_df = pd.read_csv(REG_f ,header=None,   delim_whitespace=True)
print(f'min z: {reg_df.iloc[:,2].min()}')
print(f'min lon : {reg_df.iloc[:,0].min()}')
print(f'max z: {reg_df.iloc[:,0].max()}')
print(f'max lon : {reg_df.iloc[:,2].max()}')
reg_df.columns =  ['lon', 'lat', 'classes']



if reg_df['lon'].min() >= 0 :
    reg_df['lon'] = reg_df['lon'] - 180
    


reg_df.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
reg_df.reset_index( inplace=True,drop=True)

assert reg_df['lon'].max() <= 180, f" max lon {reg_df['lon'].max()}"
assert reg_df['lon'].min() >= -180, f" min lon {reg_df['lon'].min()}"
assert reg_df['lat'].max() <= 90, f" max lat {reg_df['lat'].max()}"
assert reg_df['lat'].min() >= -90, f" min lat {reg_df['lon'].min()}"

reg_df.to_csv(REG_o , index=False, header=True, sep='\t')
reg_df.to_csv(REG_xyz_o , index=False, header=True, sep='\t')

print('terminated')

min z: 1
min lon : -180.0
max z: 180.0
max lon : 6
terminated


In [13]:
#create two datsets for A ratibng and b ratings
HGHF_file = dir_p / 'data' / 'dataset'/ 'Reference'/ 'q_Heat_flow'/'NGHF.csv'
HGHF_xyz_o_ra = dir_p / 'data' / 'dataset'/ 'XYZ'/ 'q_Heat_flow'/'NGHF_ra.xyz'
HGHF_xyz_o_rab = dir_p / 'data' / 'dataset'/ 'XYZ'/ 'q_Heat_flow'/'NGHF_rab.xyz'
HGHF_xyz_o_rabc = dir_p / 'data' / 'dataset'/ 'XYZ'/ 'q_Heat_flow'/'NGHF_rabc.xyz'
HGHF_xyz_o_rabcd = dir_p / 'data' / 'dataset'/ 'XYZ'/ 'q_Heat_flow'/'NGHF_rabcd.xyz'


hf = pd.read_csv(HGHF_file)
# change heat flow to mili

elev_cut = -1000



recorabcd_total = pd.read_csv(HGHF_file)

print(len(recorabcd_total))
hf_clean = recorabcd_total.dropna(subset = ['longitude', 'latitude', 'heat-flow (mW/m2)'])
print(len(hf_clean))

hf_no_pole = hf_clean[hf_clean['latitude'].between(world_lat_min, world_lat_max, inclusive='both')]
#hf_final = hf_no_pole[(hf_no_pole ['code6']=='A') & (hf_no_pole ['elevation (m)']>elev_cut)][['longitude', 'latitude','heat-flow (mW/m2)']]

hf_final_a = hf_no_pole[(hf_no_pole ['code6']=='A') & (hf_no_pole ['elevation (m)']>elev_cut)][['longitude', 'latitude','heat-flow (mW/m2)']]
hf_final_b = hf_no_pole[(hf_no_pole ['code6']=='B') & (hf_no_pole ['elevation (m)']>elev_cut)][['longitude', 'latitude','heat-flow (mW/m2)']]
hf_final = pd.concat([hf_final_a, hf_final_b])

hf_final_c  = hf_no_pole[(hf_no_pole ['code6']=='C') & (hf_no_pole ['elevation (m)']>elev_cut)][['longitude', 'latitude','heat-flow (mW/m2)']]
hf_final_c = pd.concat([hf_final_a, hf_final_b, hf_final_c])


hf_final_d = hf_no_pole[(hf_no_pole ['elevation (m)']>elev_cut)][['longitude', 'latitude','heat-flow (mW/m2)']]


######A rating

hf_final_a.columns = ['lon', 'lat', 'heat-flow (mW/m2)']


hf_final_a = hf_final_a.round(3)

hf_final_a.replace(' ', np.nan, inplace=True)
hf_final_a.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
hf_final_a.reset_index( inplace=True,drop=True)

assert hf_final_a['lon'].max() <= 180, f" max lon {hf_final_a['lon'].max()}"
assert hf_final_a['lon'].min() >= -180, f" min lon {hf_final_a['lon'].min()}"
assert hf_final_a['lat'].max() <= 90, f" max lat {hf_final_a['lat'].max()}"
assert hf_final_a['lat'].min() >= -90, f" min lat {hf_final_a['lon'].min()}"

hf_final_a.to_csv(HGHF_xyz_o_ra , index=False, header=True, sep='\t')

###### A+ B rating

hf_final.columns = ['lon', 'lat', 'heat-flow (mW/m2)']


hf_final = hf_final.round(3)

hf_final.replace(' ', np.nan, inplace=True)
hf_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
hf_final.reset_index( inplace=True,drop=True)
print(len(hf_final))

assert hf_final['lon'].max() <= 180, f" max lon {hf_final['lon'].max()}"
assert hf_final['lon'].min() >= -180, f" min lon {hf_final['lon'].min()}"
assert hf_final['lat'].max() <= 90, f" max lat {hf_final['lat'].max()}"
assert hf_final['lat'].min() >= -90, f" min lat {hf_final['lon'].min()}"

hf_final.to_csv(HGHF_xyz_o_rab , index=False, header=True, sep='\t')


print(len(hf_final_a))
print(len(hf_final))


###### A+ B + C rating

hf_final_c.columns = ['lon', 'lat', 'heat-flow (mW/m2)']


hf_final_c = hf_final_c.round(3)

hf_final_c.replace(' ', np.nan, inplace=True)
hf_final_c.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
hf_final_c.reset_index( inplace=True,drop=True)
print(len(hf_final))

assert hf_final_c['lon'].max() <= 180, f" max lon {hf_final_c['lon'].max()}"
assert hf_final_c['lon'].min() >= -180, f" min lon {hf_final_c['lon'].min()}"
assert hf_final_c['lat'].max() <= 90, f" max lat {hf_final_c['lat'].max()}"
assert hf_final_c['lat'].min() >= -90, f" min lat {hf_final_c['lon'].min()}"

hf_final_c.to_csv(HGHF_xyz_o_rabc , index=False, header=True, sep='\t')


###### A+ B + C rating and elevation

hf_final_d.columns = ['lon', 'lat', 'heat-flow (mW/m2)']


hf_final_d = hf_final_d.round(3)

hf_final_d.replace(' ', np.nan, inplace=True)
hf_final_d.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
hf_final_d.reset_index( inplace=True,drop=True)
print(len(hf_final))

assert hf_final_d['lon'].max() <= 180, f" max lon {hf_final_d['lon'].max()}"
assert hf_final_d['lon'].min() >= -180, f" min lon {hf_final_d['lon'].min()}"
assert hf_final_d['lat'].max() <= 90, f" max lat {hf_final_d['lat'].max()}"
assert hf_final_d['lat'].min() >= -90, f" min lat {hf_final_d['lon'].min()}"

hf_final_d.to_csv(HGHF_xyz_o_rabcd , index=False, header=True, sep='\t')



print(len(hf_final_a))
print(len(hf_final))

print(len(hf_final_c))
print(len(hf_final_d))


print('terminated')

69729
69377
12707
5792
12707
12707
12707
5792
12707
35647
46113
terminated


In [14]:
#volc_h_f =  r'./data/dataset/Reference/r_Distance_to_nearest_volcano/GVP_Volcano_List_Holocene.xlsx'
#volc_p_f = r'./data/dataset/Reference/r_Distance_to_nearest_volcano/GVP_Volcano_List_Pleistocene.xlsx'
#volc_xyz_o = f'./data/dataset/XYZ/r_Distance_to_nearest_volcano/volcanos.xyz'

    
volc_h_f =   dir_p / 'data' / 'dataset'/ 'Reference'/'c_Distance_to_nearest_volcano'/'GVP_Volcano_List_Holocene.xlsx'
volc_p_f =  dir_p / 'data' / 'dataset'/ 'Reference'/'c_Distance_to_nearest_volcano'/'GVP_Volcano_List_Pleistocene.xlsx'
volc_xyz_o =  dir_p / 'data' / 'dataset'/'XYZ'/'c_Distance_to_nearest_volcano'/'volcanos.xyz'

volc_h = pd.read_excel(volc_h_f, header=1)
volc_p = pd.read_excel(volc_p_f, header=1)
volc_df = pd.concat([volc_h, volc_p])

print(volc_df[['Latitude','Longitude']])
cols = volc_df.columns.to_list()
cols[1], cols[0] = cols[0], cols[1]
volc_df.loc[:,cols]

print(len(volc_h))
print(len(volc_p))
print(len(volc_df))

volc_final = volc_df[['Latitude','Longitude']]
volc_final.columns = [ 'lat', 'lon']
columns_titles = [ 'lon', 'lat', ]



volc_final = volc_final.round(2)
volc_final = volc_final.reindex(columns=columns_titles)

volc_final.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
volc_final.reset_index(drop=True, inplace=True)

assert volc_final['lon'].max() <= 180, f" max lon {volc_final['lon'].max()}"
assert volc_final['lon'].min() >= -180, f" min lon {volc_final['lon'].min()}"
assert volc_final['lat'].max() <= 90, f" max lat {volc_final['lat'].max()}"
assert volc_final['lat'].min() >= -90, f" min lat {volc_final['lon'].min()}"

volc_final.to_csv(volc_xyz_o , index=False, header=True, sep='\t')


print('Terminated')

      Latitude  Longitude
0       50.170      6.850
1       45.775      2.970
2       42.170      2.530
3       38.870     -4.020
4       42.600     11.930
...        ...        ...
1249    -2.228    101.430
1250    -2.418    101.776
1251    -2.480    102.026
1252    -8.416    118.207
1253   -78.374    165.015

[2652 rows x 2 columns]
1398
1254
2652
Terminated
